# Agent Scheduling Problem
This isn't the classic Traveling Salesman Problem with Time Windows because there are more constraints to respect.<br>
Those constraints are:
<ul>
    <li>30 minutes for lunch between 12:00 AM and 2:00 PM;</li>
    <li>Limited working time (8 hours);</li>
    <li>Limited waiting and traveling time;</li>
    <li>Minimum time to spent in office (1 hour);</li>
</ul>

 #### Importing libraries

In [1]:
import gurobipy as gb
from gurobipy import GRB

import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

#### Define file with clients data

In [2]:
FILE = "./TEST_SETS/test_1.txt"

#### Define costant parameters

In [3]:
# Default params
SUPPORTED_FORMAT = ['NUM', 'X', 'Y', 'DEMAND', 'READYTIME', 'DUEDATE', 'SERVICE']
# Macros for time values conversions
MINUTES = 60
HOURS = 3600
OFFSET_TIMES = 8*HOURS

COLUMNS_OPS = {'NUM': lambda x: float(x),
               'X': lambda x: float(x),
               'Y': lambda x: float(x),
               'DEMAND': lambda x: 1,
               'READYTIME': lambda x: float(x),
               'DUEDATE': lambda x: float(x),
               'SERVICE': lambda x: float(x)
              }
# Agents count
AGENTS = 10

# Multiplier for distance cost
TIME_PER_DISTANCE = 1

# Agent Working day start and end
WORKING_TIME_RANGE = (0, 8*HOURS)

# Agent Lunch break time range, lasting
LUNCH_BREAK_RANGE = (12*HOURS-OFFSET_TIMES, 13.5*HOURS-OFFSET_TIMES)
LUNCH_BREAK_TIME = 30*MINUTES

# Agents office parameters
OFFICE_NUM = 0
OFFICE_X = .0
OFFICE_Y = .0
OFFICE_READYTIME = WORKING_TIME_RANGE[0]
OFFICE_DUEDATE = WORKING_TIME_RANGE[1]
OFFICE_SERVICE = 1*HOURS

#### Read clients data

In [4]:
def read_input_tsptw(filename):
    """This function is used to convert input file to usable data"""
    # Dict sed for locations parameters
    data_dict = dict()
    
    # List of node positions for plots
    nodes_x = list()
    nodes_y = list()
       
    # Add office to data
    data_dict.update({OFFICE_NUM: {'X': OFFICE_X, 
                                   'Y': OFFICE_Y, 
                                   'DEMAND': AGENTS,
                                   'READYTIME': OFFICE_READYTIME,
                                   'DUEDATE': OFFICE_DUEDATE,
                                   'SERVICE': OFFICE_SERVICE,}})
    # Add office to nodes
    nodes_x.append(OFFICE_X)
    nodes_y.append(OFFICE_Y)
    
    # Open file and read lines 
    with open(filename, "r") as file:
        # Initialize columns in empty dict
        columns = file.readline().replace("#","").split()
        if columns != SUPPORTED_FORMAT:
            print("ERROR! Format not supported.")
            return 
                   
        # For each data line
        for line in file.readlines():
            node_dict = {k: COLUMNS_OPS[k](val) for k, val in zip(columns, line.split())}
                
            # Get id
            node_id = node_dict.pop('NUM')
            # Insert new node in data dict
            data_dict.update({int(node_id): node_dict})            
            # Get nodes positions
            nodes_x.append(float(line.split()[columns.index('X')]))
            nodes_y.append(float(line.split()[columns.index('Y')]))

    # Get distance matrix
    distance_matrix = compute_distance_matrix(nodes_x, nodes_y)
    return (data_dict, distance_matrix, dict(enumerate(zip(nodes_x, nodes_y))))


def compute_distance_matrix(nodes_x, nodes_y):
    """This function is used to compute the distance matrix"""
    # Get clients count and initialize distance matrix
    clients = len(nodes_x)
    distance_matrix = [[None for i in range(clients)] for j in range(clients)]
    for i in range(clients):
        # Set cost of trip between same agent and himself as null
        distance_matrix[i][i] = 0
        for j in range(clients):
            # Compute distance matrix calculating euclidean distance between each node
            dist = compute_dist(nodes_x[i], nodes_x[j], nodes_y[i], nodes_y[j])
            distance_matrix[i][j] = dist
            distance_matrix[j][i] = dist
    return distance_matrix


def compute_dist(xi, xj, yi, yj):
    """This function is used to compute euclidean distance"""
    exact_dist = math.sqrt(math.pow(xi - xj, 2) + math.pow(yi - yj, 2))
    return int(math.floor(exact_dist + 0.5)) * TIME_PER_DISTANCE

In [5]:
# Getting locations parameters
data_dict, distance_matrix, positions = read_input_tsptw(FILE)

In [6]:
# DEBUG RESTRICTIONS
CLIENTS = len(data_dict)
data_dict = {k: v for k,v in data_dict.items() if k < CLIENTS}
distance_matrix = [dm[:CLIENTS] for dm in distance_matrix[:CLIENTS]]

# ADD FITTICIOUS LOCATION
# This location is used to have a complete loop in Agent trips without interfering 
# with trips costs. Having a complete loop simplify the job of creating a trip.
# To not interfer with costs it's distance to all other locations is 0. 
distance_matrix = [dm + [0,] for dm in distance_matrix]
distance_matrix = distance_matrix + [[0]*(CLIENTS+1)]
# Add location data
data_dict.update({CLIENTS: {'X': 0, 
                            'Y': 0, 
                            'DEMAND': AGENTS,
                            'READYTIME': WORKING_TIME_RANGE[0],
                            'DUEDATE': WORKING_TIME_RANGE[1],
                            'SERVICE': 0,}})

# POSITIONS SETS FOR CLEANER MODEL
agent_list = list(range(AGENTS))
all_pos = list(range(CLIENTS*2))
start_pos = CLIENTS
client_pos = list(range(1,CLIENTS))
only_start_office_pos = 0
no_duplicates_pos = list(range(0,CLIENTS))
destination_office_pos = list(range(CLIENTS+1, CLIENTS*2))
office_pos = [only_start_office_pos]+destination_office_pos
reachable_pos = [p for p in range(CLIENTS*2) if p not in [start_pos,]]

#--------------
clients_list = list(range(CLIENTS))
meets_duedates = [(i, data_dict[i]['READYTIME']) for i in data_dict.keys()]

In [7]:
'''this fun trasforms the dict in a tuplelist [(key, schedule) .. ] 
example [(1,[(office)(meet4)..]), (1,[(meet1)(meet2)..]), (2,[(office)(meet7)..])]
for each value in this tuplelist will be assigned a binary variable '''

def totuplelist(scheds):#TODO convertire per un semplice dict e non l'oggetto sa
    schedules = gb.tuplelist() #provare a usare tupledict
    for agent in scheds.keys():
        for sched in scheds[agent]:
            #print(sched)
            schedules.append((agent, sched))
    return schedules

def path_cost(path):
    p = path.copy()
    p.append(('end', WORKING_TIME_RANGE[1], -1)) #dummy at end
    cost = 0
    for i in range(len(p)-1):
        time_range = p[i+1][1] - p[i][1]
        if p[i][0] == 'wait' or p[i][0] == 'travel': #travel?
            cost = cost + time_range
    return cost

## Models

### Primal master problem

In [8]:
def pmp(schedules):#qui passo il dict
    #Constants
    scheds_list = list(range(len(schedules)))
    meets_list = list(range(len(meets_duedates)))
    positions = [m[0] for m in meets_duedates]
    
    # Create model
    mod = gb.Model("TSPTW")

    #Vars
    #x = mod.addVars(scheds_list, name='x', vtype=GRB.BINARY)
    x = mod.addVars({(a,s): 0 for a in schedules.keys()
                          for s in range(len(schedules[a])) },
               name="x",
               vtype=GRB.BINARY)
    
    #Constrs
    oneschedperagent = mod.addConstrs((gb.quicksum(x[a,s] for s in range(len(schedules[a]))) == 1 for a in agent_list),
                                      name='one_sched_per_agent')
    
    meetsconstr = mod.addConstrs((gb.quicksum(x[a,s]
                                              for a in agent_list 
                                              for s in range(len(schedules[a])) 
                                              if pos in [work[2] for work in schedules[a][s] if work[0] == 'meet'])
                                              == 1 for pos in positions), name='meets_constr')

    #Obj
    mod.setObjective((gb.quicksum(x[a,s] * path_cost(schedules[a][s])
                                  for a in agent_list
                                  for s in range(len(schedules[a])))), GRB.MINIMIZE)
    
    return mod

### Relaxed primal master problem

In [9]:
def rpmp(schedules):
    #Constants
    scheds_list = list(range(len(schedules)))
    meets_list = list(range(len(meets_duedates)))
    
    H = 100000
    # Create model
    mod = gb.Model("Relaxed_TSPTW")

    #Vars
    x = mod.addVars(schedules, name="x", vtype=GRB.CONTINUOUS)
    y = mod.addVars(meets_duedates, name='y', vtype=GRB.CONTINUOUS)
    
    #Constrs
    oneschedperagent = mod.addConstrs((x.sum(a,'*') == 1 for a in agent_list), name='one_sched_per_agent')
    meetsconstr = mod.addConstrs((gb.quicksum(x[i] for i in scheds_list
                                               if meets_duedates[m][0] in [work[2] for work in schedules[i][1]
                                                                           if work[0] == 'meet']) + 
                                 gb.quicksum(y[m]) == 1 for m in meets_list), name = 'meets_constr')
    
    nonnegativex = mod.addConstrs((x[s] >= 0 for s in scheds_list), name = 'non_negative_x')
    nonnegativey = mod.addConstrs((y[m] >= 0 for m in meets_list), name = 'non_negative_y')
    
    #Obj
    mod.setObjective(quicksum(x[i] * path_cost(schedules[i][1]) for i in scheds_list) + (quicksum(y[m] for m in meets_list) * H), GRB.MINIMIZE)
    
    return mod

### Define dual relaxed master problem

In [10]:
def drmp(schedules):
    #Constants
    scheds_list = list(range(len(schedules)))
    meets_list = list(range(len(meets_duedates)))
    
    H = 10000
    #Create model
    mod = gb.Model("Dual_TSPTW")
    
    #Variables
    w = mod.addVars(agent_list, name='w', vtype=GRB.CONTINUOUS)
    z = mod.addVars(meets_duedates, name='z', vtype=GRB.CONTINUOUS)
    y = mod.addVars(meets_duedates, name='y', vtype=GRB.CONTINUOUS)
    
    #Constraints
    _ = mod.addConstrs(y[m] < H for m in meets_list)
    _ = mod.addConstrs(w[s[0]] + gb.quicksum(z[m] for m in meets_list if m in [work[2] for work in s[1] if work[0] == 'meet']) <= path_cost(s[1]) 
                       for s in schedules) #s[0] = agent, s[1] = path [work0,work1..] work: (type, time, meet_loc) 
    
    #Obj
    mod.setObjective(quicksum(w[a] for a in agent_list) + quicksum(z[m] for m in meets_list), GRB.MAXIMIZE)
    
    return mod

In [11]:
'''Partendo da una schedula per ogni agente, genera tutte le schedule che si ottengono scambiando due nodi tra loro'''
def swap(agents_paths, swap3=False):
    lens = { i : len(agents_paths[i]) for i in range(len(agents_paths))}
    nodes = []
    for i in range(len(agents_paths)):
        nodes = nodes + agents_paths[i]
    all_paths = []
    #swap 2
    for i in range(len(nodes)-1):
        for j in range(i+1, len(nodes)):
            new_nodes = nodes.copy()
            (new_nodes[i], new_nodes[j]) = (new_nodes[j], new_nodes[i])
    
            counter = 0
            for k in range(len(agents_paths)):
                new_path = []
                new_path = new_path + new_nodes[counter:counter+lens[k]]
                counter = counter + lens[k]
                agents_paths[k] = new_path
            all_paths.append(agents_paths.copy())
            
    #swap 3
    if swap3:
        for i in range(len(nodes)-2):
            for j in range(i+1, len(nodes)-1):
                for h in range(j+1, len(nodes)):
                    new_nodes = nodes.copy()
                    (new_nodes[i], new_nodes[j], new_nodes[h]) = (new_nodes[h], new_nodes[i], new_nodes[j])
    
                    counter = 0
                    for k in range(len(agents_paths)):
                        new_path = []
                        new_path = new_path + new_nodes[counter:counter+lens[k]]
                        counter = counter + lens[k]
                        agents_paths[k] = new_path
                    all_paths.append(agents_paths.copy())
    return all_paths

def all_schedule_comb(agents_paths):
    scheds = paths_to_schedule()
    all_combinations = swap(agents_paths)
    for comb in all_combinations: #all_combination is a list of dicts
        for key in comb.keys():
            #print(key, comb[key])
            sched = path_to_schedule(comb[key])
            if is_sched_feasible(sched) and sched not in scheds[key]:
                scheds[key].append(sched)
                
    return scheds

def schedules_from_mod(mod):
    l = []
    for v in mod.getVars():
        if v.x == 1:
            s = str(v.varName)
            s = s[1:len(s)]
            split = s.split(',')
            cmd = 'l.append(scheds'+split[0]+']['+split[1]+')'
            exec(cmd)

    scheds = {i: l[i] for i in range(len(l))}
    agents_paths = {i: [w[2] for w in l[i] if w[0]=='meet'] for i in range(len(l))}
    
    return scheds

In [12]:
'''
Per ogni agente cerco il percorso migliore per seguire più meeting possibili, considero anche il pranzo
'''
#TODO: 0 E 20 vanno tolti dal dict perchè indicano l'ufficio e non un meet
#init
agents_paths = {a:[] for a in agent_list} #-1?
uncovered_meets = meets_duedates.copy()
uncovered_meets.sort(key=lambda x: x[1])
for i in range(len(agents_paths)):
    lunch_done = False
    if uncovered_meets:
        next_meet = uncovered_meets.pop(0)
        freeat = next_meet[1] + data_dict[next_meet[0]]['SERVICE']
        if next_meet[1] <= LUNCH_BREAK_RANGE[0] and freeat >= LUNCH_BREAK_RANGE[1]:
            raise Exception("Inizializzazione fallita: un meet occupa tutta la durata di un pranzo.")
            
        agents_paths[i].append(next_meet[0])
        if next_meet[1] >= LUNCH_BREAK_RANGE[0] + LUNCH_BREAK_TIME and next_meet[1] <= LUNCH_BREAK_RANGE[1] + LUNCH_BREAK_TIME: #ho già pranzato
            lunch_done = True
        elif freeat >= LUNCH_BREAK_RANGE[0] and freeat <= LUNCH_BREAK_RANGE[1]: #è ora di pranzo: sposto freeat di mezz'ora
            freeat = freeat + LUNCH_BREAK_TIME
            lunch_done = True
        
        to_remove = []
        for m in uncovered_meets:
            if m[1] <= LUNCH_BREAK_RANGE[0] and m[1] + data_dict[m[0]]['SERVICE'] >= LUNCH_BREAK_RANGE[1]: 
                raise Exception("Inizializzazione fallita: un meet occupa tutta la durata di un pranzo.")
                
            if freeat + distance_matrix[agents_paths[i][-1]][m[0]]*MINUTES <= m[1]: #TODO sostituire MINUTES con TIMEPERDISTANCE
                freeat = m[1] + data_dict[m[0]]['SERVICE']
                agents_paths[i].append(m[0])
                to_remove.append(m)
                if m[1] >= LUNCH_BREAK_RANGE[0] + LUNCH_BREAK_TIME and m[1] <= LUNCH_BREAK_RANGE[1] + LUNCH_BREAK_TIME and not lunch_done: #ho già pranzato
                    lunch_done = True #m[1] - travel?
                if freeat >= LUNCH_BREAK_RANGE[0] and freeat <= LUNCH_BREAK_RANGE[1] and not lunch_done: #è ora di pranzo: sposto freeat di mezz'ora
                    freeat = freeat + LUNCH_BREAK_TIME
                    lunch_done = True
        
        for m in to_remove:
            uncovered_meets.remove(m)

'''Converte una lista di nodi (path) in una schedula. Ogni nodo rappresenta la sequenza dei meet, e se c'è
abbastanza tempo si torna in ufficio'''

def path_to_schedule(path):
    sched = []
    if not path:
            sched.append(('office', 0.0, 0))
            sched.append(('lunch', LUNCH_BREAK_RANGE[0], -1))
            sched.append(('office', LUNCH_BREAK_RANGE[0]+LUNCH_BREAK_TIME, 0))
    else:
        lunch_done = False
        lunch_shift = False
        meet_hour = data_dict[path[0]]['READYTIME']
        meet_end = meet_hour + data_dict[path[0]]['SERVICE']
        if OFFICE_SERVICE + distance_matrix[0][path[0]]*MINUTES <= meet_hour:
            start_travel =  meet_hour - distance_matrix[0][path[0]]*MINUTES
            sched.append(('office', 0.0, 0))
            if meet_hour >= LUNCH_BREAK_RANGE[0] + LUNCH_BREAK_TIME and meet_hour <= LUNCH_BREAK_RANGE[1] + LUNCH_BREAK_TIME and not lunch_done:
                sched.append(('travel', start_travel - LUNCH_BREAK_TIME, -1)) #travel e lunch sono scambiabili, non fa differenza se
                sched.append(('lunch', meet_hour - LUNCH_BREAK_TIME, -1))     # se l'agente prima viaggia e poi mangia o viceversa
                lunch_done = True
            else: 
                sched.append(('travel', start_travel, -1))
            sched.append(('meet', meet_hour, path[0]))
        else:
            sched.append(('wait', 0.0, -1))
            sched.append(('meet', meet_hour, path[0]))
        if meet_end >= LUNCH_BREAK_RANGE[0] and meet_end <= LUNCH_BREAK_RANGE[1] and not lunch_done:
            sched.append(('lunch', meet_end, -1)) # in questo caso parto per il prossimo meet mezz'ora dopo
            #freeat traslato?
            lunch_done = True
        
        #iter lungo la lista i, i+1 fino a n-1
        for j in range(len(path)-1):
            prev = path[j]
            _next = path[j+1]
            #diff = data_dict[_next]['READYTIME'] - data_dict[prev]['SERVICE'] - data_dict[prev]['READYTIME'] #CAMBIATO _next in prev ['SERVICE']
            shift = 0.0
            if lunch_shift:
                shift = LUNCH_BREAK_TIME
                lunch_shift = False
            end_prev_meet = data_dict[prev]['READYTIME'] + data_dict[prev]['SERVICE'] + shift
            start_next_meet = data_dict[_next]['READYTIME']
            end_next_meet = start_next_meet + data_dict[_next]['SERVICE']
            diff = start_next_meet - end_prev_meet
            
            if ((start_next_meet >= LUNCH_BREAK_RANGE[0] + LUNCH_BREAK_TIME #è ora di pranzo
                and start_next_meet <= LUNCH_BREAK_RANGE[1] + LUNCH_BREAK_TIME) or
                (end_next_meet >= LUNCH_BREAK_RANGE[0] and end_next_meet <= LUNCH_BREAK_RANGE[1])) and not lunch_done:
                #Check se pranzare prima del meet
                if start_next_meet >= LUNCH_BREAK_RANGE[0] + LUNCH_BREAK_TIME and start_next_meet <= LUNCH_BREAK_RANGE[1] + LUNCH_BREAK_TIME and not lunch_done: #check prima del meet
                    #tempo per tornare in ufficio
                    if OFFICE_SERVICE + distance_matrix[0][prev]*MINUTES + distance_matrix[0][_next]*MINUTES + LUNCH_BREAK_TIME <= diff:
                        in_office = end_prev_meet + distance_matrix[0][prev]*MINUTES
                        start_travel_to_meet = start_next_meet - distance_matrix[0][_next]*MINUTES - LUNCH_BREAK_TIME #pranzo all'arrivo
                        sched.append(('travel', end_prev_meet, -1))
                        sched.append(('office', in_office, 0))
                        sched.append(('travel', start_travel_to_meet, -1))
                        sched.append(('lunch', start_next_meet - LUNCH_BREAK_TIME, -1))
                        sched.append(('meet', start_next_meet, _next))
                        lunch_done = True
                    else:
                        #per l'inizializzazione si è tenuto conto che wait sia maggiore di LUNCH_BREAK_TIME (SOLO INIT)
                        travel_arrive = end_prev_meet + distance_matrix[prev][_next]*MINUTES
                        lunch_diff = start_next_meet - travel_arrive
                        if lunch_diff >= LUNCH_BREAK_TIME:
                            sched.append(('travel', end_prev_meet, -1))
                            sched.append(('lunch', travel_arrive, -1))
                            sched.append(('wait', travel_arrive + LUNCH_BREAK_TIME, -1))
                            sched.append(('meet', start_next_meet, _next))
                            lunch_done = True
                        else: #è ora di pranzo ma non c'è abbastanza tempo #(non dovrebbe accadere con l'init)
                            sched.append(('travel', end_prev_meet, -1))
                            sched.append(('wait', travel_arrive, -1))
                            sched.append(('meet', start_next_meet, _next))
                #Check se il pranzo sta dopo il meet         
                if end_next_meet >= LUNCH_BREAK_RANGE[0] and end_next_meet <= LUNCH_BREAK_RANGE[1] and not lunch_done: #check fine meet
                    if OFFICE_SERVICE + distance_matrix[0][prev]*MINUTES + distance_matrix[0][_next]*MINUTES <= diff:
                        in_office = end_prev_meet + distance_matrix[0][prev]*MINUTES
                        start_travel_to_meet = start_next_meet - distance_matrix[0][_next]*MINUTES
                        sched.append(('travel', end_prev_meet, -1))
                        sched.append(('office', in_office, 0))
                        sched.append(('travel', start_travel_to_meet, -1))
                        sched.append(('meet', start_next_meet, _next))
                        sched.append(('lunch', end_next_meet, -1)) #prossima iterazione end_prev_meet swifta
                        lunch_done = True
                        lunch_shift = True
                    else:#non c'è tempo per tornare in ufficio
                        travel_arrive = end_prev_meet + distance_matrix[prev][_next]*MINUTES
                        if _next == path[-1]: #if _next+1 == len(path): #caso particolare in fondo
                            sched.append(('travel', end_prev_meet, -1))
                            sched.append(('wait', travel_arrive, -1))
                            sched.append(('meet', start_next_meet, _next))
                            sched.append(('lunch', end_next_meet, -1))
                            lunch_shift = True
                            lunch_done = True
                        else:
                            lunch_diff = data_dict[path[j+2]]['READYTIME'] - end_next_meet
                            if lunch_diff >= LUNCH_BREAK_TIME:#check diff
                                sched.append(('travel', end_prev_meet, -1))
                                sched.append(('wait', travel_arrive, -1))
                                sched.append(('meet', start_next_meet, _next))
                                sched.append(('lunch', end_next_meet, -1))
                                lunch_shift = True
                                lunch_done = True
                            else:
                                sched.append(('travel', end_prev_meet, -1))
                                sched.append(('wait', travel_arrive, -1))
                                sched.append(('meet', start_next_meet, _next)) #è ora di pranzo ma non c'è tempo
                    
            else:#non è ora di pranzo
                if OFFICE_SERVICE + distance_matrix[0][prev]*MINUTES + distance_matrix[0][_next]*MINUTES <= diff: #se c'è tempo per tornare in ufficio
                    in_office = end_prev_meet + distance_matrix[0][prev]*MINUTES
                    start_travel_to_meet = start_next_meet - distance_matrix[0][_next]*MINUTES
                    sched.append(('travel', end_prev_meet, -1))
                    sched.append(('office', in_office, 0))
                    sched.append(('travel', start_travel_to_meet, -1))
                    sched.append(('meet', start_next_meet, _next))
                else:
                    #travel -> wait -> meet
                    waiting = end_prev_meet + distance_matrix[prev][_next]*MINUTES
                    sched.append(('travel', end_prev_meet, -1))
                    sched.append(('wait', waiting, -1))
                    sched.append(('meet', start_next_meet, _next))
        #finale
        last = path[len(path)-1] #per questo meet prima o dopo il pranzo se c'era possibilità è stato piazzato
        shift = 0.0
        if lunch_shift:
            shift = LUNCH_BREAK_TIME
            lunch_shift = False
        last_meet_end = data_dict[last]['READYTIME'] + data_dict[last]['SERVICE'] + shift #se prima c'è stato il pranzo considero lo shift
        office_arriving = last_meet_end + distance_matrix[0][last]*MINUTES #TODO TIMEPERDISTANCE
        if WORKING_TIME_RANGE[1] - office_arriving >= OFFICE_SERVICE:
            sched.append(('travel', last_meet_end, -1))
            sched.append(('office', office_arriving, 0))
        else:
            sched.append(('wait', last_meet_end, -1)) #fine della giornata
            
    #check lunch num
    lunch_num = len([m for m in sched if m[0] == 'lunch'])
    if lunch_num > 1:
        raise Exception('Errore: più di un pranzo al giorno in una schedula: ' + str(path) + ' --- ' + str(sched))
    elif lunch_num == 0:#cerco di incastrare il pranzo se c'è del tempo in ufficio che mi permette di farlo
        sched.append(('end', WORKING_TIME_RANGE[1], -1)) #rimuovo end alla fine
        for i in range(len(sched)-1):
            r1=sched[i][1]
            r2=sched[i+1][1]
            if sched[i][0] == 'office':
                if (r1 <= LUNCH_BREAK_RANGE[0] and r2 >= LUNCH_BREAK_RANGE[0] + LUNCH_BREAK_TIME):
                    sched.insert(i+1, ('lunch', LUNCH_BREAK_RANGE[0], -1))
                    sched.insert(i+2, ('office', LUNCH_BREAK_RANGE[0] + LUNCH_BREAK_TIME, 0))
                    break
                elif r1 >= LUNCH_BREAK_RANGE[0] and r2 <= LUNCH_BREAK_RANGE[1] and r2-r1 >= LUNCH_BREAK_TIME:
                    sched.insert(i+1, ('lunch', r1, -1))
                    sched.insert(i+2, ('office', r1 + LUNCH_BREAK_TIME, -1))
                    del(a[i])#sostituisco office con lunch
                    break
                elif r1 <= LUNCH_BREAK_RANGE[1] and r2 >= LUNCH_BREAK_RANGE[1] + LUNCH_BREAK_TIME:
                    sched.insert(i+1, ('lunch', LUNCH_BREAK_RANGE[1], -1))
                    sched.insert(i+2, ('office', LUNCH_BREAK_RANGE[1] + LUNCH_BREAK_TIME, 0))
                    break
        del(sched[-1])#rimuovo end
    return sched

#convert path in schedule (dove ci sono dei buchi con abbastanza tempo per tornare in ufficio ci torno)
def paths_to_schedule():
    #ho una lista di meet in sequenza, devo "incastrare" ritorni in ufficio e pranzi
    scheds = {a: [] for a in agent_list}
    for i in agent_list:
        scheds[i].append(path_to_schedule(agents_paths[i]))
    return scheds

def is_sched_feasible(schedule):#scorro la schedula e se c'è un valore < del precedente torno false, altrimenti true
    #verifico che ci sia un solo pranzo
    '''
    lunch_num = len([w for w in schedule if w[0] == 'lunch'])
    if lunch_num != 1:
        return False
    '''
    time = 0.0
    for work in schedule:
        if time > work[1]:
            #print(work)
            return False
        time = work[1]
    return True        

## Model optimization

In [13]:
#initial cost
last_cost = 0
scheds = paths_to_schedule()
#nodes = {0:[7,6,19,5,20,10,15,4,1], 1:[8,11,2], 2:[14,13,12,17,3,9,18,16]}
#agents_paths = nodes
#scheds = paths_to_schedule()
print(scheds)

for key in scheds.keys():
    last_cost = last_cost + path_cost(scheds[key][0])
print("Initial cost: " + str(last_cost))

while (True):
    scheds = all_schedule_comb(agents_paths)
    #print(scheds)
    mod = pmp(scheds)
    mod.optimize()
    
    scheds = schedules_from_mod(mod)#also updates agents_paths for next iteration

    cost = 0
    for key in scheds.keys():
        cost = cost + path_cost(scheds[key])
    print(last_cost, cost)
    if (last_cost <= cost):
        break
    else:
        last_cost = cost

{0: [[('wait', 0.0, -1), ('meet', 0, 0), ('travel', 3600.0, -1), ('wait', 5280.0, -1), ('meet', 8400.0, 2), ('travel', 10800.0, -1), ('lunch', 13320.0, -1), ('wait', 15120.0, -1), ('meet', 18700.0, 16), ('travel', 21100.0, -1), ('wait', 23260.0, -1), ('meet', 24800.0, 1), ('wait', 27200.0, -1)]], 1: [[('wait', 0.0, -1), ('meet', 0.0, 14), ('travel', 2400.0, -1), ('wait', 4020.0, -1), ('meet', 4800.0, 12), ('travel', 7200.0, -1), ('wait', 11580.0, -1), ('meet', 12100.0, 9), ('lunch', 14500.0, -1), ('travel', 16300.0, -1), ('wait', 18520.0, -1), ('meet', 20000.0, 15), ('wait', 22400.0, -1)]], 2: [[('wait', 0.0, -1), ('meet', 0, 21), ('travel', 0.0, -1), ('wait', 0.0, -1), ('meet', 2400.0, 13), ('travel', 4800.0, -1), ('wait', 8040.0, -1), ('meet', 8500.0, 19), ('travel', 10900.0, -1), ('wait', 12700.0, -1), ('meet', 13300.0, 20), ('lunch', 15700.0, -1), ('travel', 17500.0, -1), ('wait', 21640.0, -1), ('meet', 22400.0, 4), ('wait', 24800.0, -1)]], 3: [[('wait', 0.0, -1), ('meet', 3600.0, 

In [14]:
print(scheds)

{0: [('wait', 0.0, -1), ('meet', 0, 0), ('travel', 3600.0, -1), ('wait', 5280.0, -1), ('meet', 8400.0, 2), ('travel', 10800.0, -1), ('lunch', 12360.0, -1), ('wait', 14160.0, -1), ('meet', 20000.0, 15), ('travel', 22400.0, -1), ('wait', 23120.0, -1), ('meet', 24800.0, 1), ('wait', 27200.0, -1)], 1: [('wait', 0.0, -1), ('meet', 0.0, 14), ('travel', 2400.0, -1), ('wait', 4020.0, -1), ('meet', 4800.0, 12), ('travel', 7200.0, -1), ('wait', 11580.0, -1), ('meet', 12100.0, 9), ('lunch', 14500.0, -1), ('travel', 16300.0, -1), ('wait', 18280.0, -1), ('meet', 18700.0, 16), ('travel', 21100.0, -1), ('office', 24400.0, 0)], 2: [('wait', 0.0, -1), ('meet', 0, 21), ('travel', 0.0, -1), ('wait', 0.0, -1), ('meet', 3700.0, 7), ('travel', 6100.0, -1), ('wait', 6340.0, -1), ('meet', 8500.0, 19), ('travel', 10900.0, -1), ('wait', 12700.0, -1), ('meet', 13300.0, 20), ('lunch', 15700.0, -1), ('travel', 17500.0, -1), ('wait', 21640.0, -1), ('meet', 22400.0, 4), ('wait', 24800.0, -1)], 3: [('wait', 0.0, -1),